In [1]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
from io import BytesIO
import json


import http.client, urllib.request, urllib.parse, urllib.error, base64

In [2]:
# Define General Global Variables
GROUP_NAME = "hackathon_atl"

# Define Face-API Specific Global Variables
FACE_API_KEY1 = "2d697fcd49e34ceaacce26f321034221"
FACE_API_URL = 'https://eastus2.api.cognitive.microsoft.com/face/v1.0'
HEADERS = {
    'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    'Content-Type': 'application/octet-stream'
}
PARAMS = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'true',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,' +
    'emotion,hair,makeup,occlusion,accessories,blur,exposure,noise'
}

# 1. Create the Person Group

In [3]:
def createPersonGroup():
    PARAMS = urllib.parse.urlencode({
        'personGroupId': GROUP_NAME
    })
    BODY = '{"name": "%s"}' % GROUP_NAME

    try:
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("PUT", "/face/v1.0/persongroups/{%s}?%s" % (GROUP_NAME, PARAMS), "%s" % (BODY), HEADERS)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

# 2. Insert People into Person Group

In [4]:
def insertPersonIntoGroup(person_name, group_name):
    HEADERS = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    
    PARAMS = urllib.parse.urlencode({
        'personGroupId': group_name
    })
    
    BODY = '{"name":"%s"}' % (person_name)
    
    try:
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/persongroups/{%s}/persons?%s" % (group_name, PARAMS), "%s" % (BODY), HEADERS)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
        return data
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return None
    
insertPersonIntoGroup("Obama", GROUP_NAME)

b'{"personId":"8c6da2d7-5929-4b4b-873b-fa7fa6f595f2"}'


b'{"personId":"8c6da2d7-5929-4b4b-873b-fa7fa6f595f2"}'

# 3. Submit Images to Train Face On

In [8]:
Obama_ID = "f9a6fb5d-2c81-40c0-9cb0-e9980d62371c"

import os
#for filename in os.listdir('.\\Obama'):
#    print(filename)

image_data = open("./obama/IMG_0909_001.jpg", "rb").read()
str_data = str(base64.b64decode(image_data))
str_data

'b\'$R\\x05-\\xab\\xdc\\xe7\\xcd|\\xd7M\\x02\\xfb\\xed\\xf5\\x01\\n\\x9a\\xd9\\x1a\\xdb\\x07p\\x92\\xe3\\x9b-Om@A\\xaa\\xa1X\\x1d\\x00\\xac&\\xad\\xb7\\x8bE\\xf3\\xd3\\xd0\\x10hVX8E\\x17F\\xb4\\xae\\xd0-X\\x10t\\xb4F\\xad\\x0b0\\xf4\\xc9\\xcc\\xc0\\xd2\\xb5\\xc4@*\\x83:_V\\x0c\\xed\\x04\\x08\\x10\\xf5\\x10v\\xe6\\t\\xe3\\xf4>e\\x8a\\x91\\xb7\\x0b+1\\xccw\\x17]\\xd0Nw\\x1c%\\xf6\\x0bn\\x1fu\\xb7\\x8bo\\x80\\xe1\\xd8\\x8dB\\x1c\\x8e\\xf5+o\\xc9D\\x80\\xa2\\xd9\\x12\\x9b\\xcf\\\'\\x0b\\xbbC\\xe6\\x9a\\xf9\\x9c\\x8a\\xe5\\xa7\\xeb\\xa8\\xb3\\xae\\x9a\\xdc\\xa8e\\xde\\x8c&@\\x0fTaG}\\xc5t0\\xf6\\x11\\xd0\\xc73\\x81{\\xdd\\xde=[\\xdc\\xd9\\xa9\\xe7\\xd1\\x1a\\xcaH\\x80\\tc\\x98\\xb6\\xc8Y\\xf2\\r\\x1bZ\\xa45\\xda{~\\xb3\\xb6\\xcf\\xbd\\xcf\\xc8\\xa4\\xff\\x7f\\xda\\x9d\\xa6(\\xb7\\xd3]7\\xe1\\xf2\\xcf\\xd8\\x0e6CXI\\x9a\\xc1\\xbbB|\\xa4?I$\\xddoY\\xac\\xad\\xabV\\x07\\xee=\\x8b(\\xfeR\\xaeNF\\x03\\t\\x89\\x9b\\xd3yH\\x0c\\x8e\\xb6\\x1c0\\xd3\\xd9\\t\\x8a\\x06\\x1b\\x0c\\x19\\xc7%\\x99\\\'\\x

In [9]:
def submitPersonImages(person_id, image_data):
    # Declare header data
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    
    # Obtain full URL Path
    path = FACE_API_URL + "/persongroups/%s/persons/%s/persistedFaces" % (GROUP_NAME, person_id)
    
    try:
        response = requests.post(path, headers=headers, data=image_data)
        faces = response.json()
        print(faces)
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
# Loop through Obama picture directory.
img_path = './obama/'
for filename in os.listdir(img_path):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        image_data = open(img_path + filename, "rb").read()
        submitPersonImages(Obama_ID, image_data)

{'persistedFaceId': 'a84d0627-c084-48ee-b1f7-1d72dbd71884'}


# 4. Train Person Group

In [11]:
def trainPersonGroup():   
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }

    try:
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/persongroups/%s/train" % (GROUP_NAME), "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return
        
trainPersonGroup()

b''


# 5. Identify Obama

In [12]:
def detectFace(image_data):
    # Declare header data
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
   #detect[?returnFaceId][&returnFaceLandmarks][&returnFaceAttributes]
    params = ({
        # Request parameters
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'false',
        'returnFaceAttributes': 'emotion',
    })
    
    path = FACE_API_URL + '/detect'
    
    try:
        response = requests.post(path, headers=headers, params=params, data=image_data)
        faces = response.json()
        #print(faces)
        return faces
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return None

In [13]:
def identifyPerson(face_id):
    # Declare header data
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    
    body = {
        "personGroupId": GROUP_NAME,
        "faceIds": [face_id],
        "maxNumOfCandidatesReturned": 1,
        "confidenceThreshold": 0.5
    }
    
    # Obtain full URL Path
    path = FACE_API_URL + "/identify"
    
    try:
        response = requests.post(path, headers=headers, data=json.dumps(body))
        faces = response.json()
        #print(faces)
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [14]:
def getFaceIdentification(image):
    image_data = open(image, "rb").read()
    face_detection = detectFace(image_data)
    face_identity = identifyPerson(face_detection[0]['faceId'])
    return face_detection, face_identity
    
# Loop through Obama picture directory.
img_path = './obama/'
for filename in os.listdir(img_path):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        print(getFaceIdentification(img_path + filename))

([{'faceId': 'c4707421-fe8b-4025-b674-93b09d7989a8', 'faceRectangle': {'top': 74, 'left': 278, 'width': 101, 'height': 101}, 'faceAttributes': {'emotion': {'anger': 0.0, 'contempt': 0.005, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.026, 'neutral': 0.968, 'sadness': 0.001, 'surprise': 0.0}}}], None)


In [ ]:
#image_folder_path = "C:\\hackathon\\voicepassword\\IMG_0909\\IMG_0909_012.jpg"
#image_data = open(image_path, "rb").read()